# Analyze

## Load files

- I90DIA27: Bilateral

In [1]:
import pandas as pd

df_base = pd.read_excel(
    'data/I90DIA/I90DIA_20240221.xls',
    sheet_name='I90DIA27', skiprows=3, index_col=[0,1,2]
).iloc[:,2:]

df_base

,,,00-01,01-02,02-03,03-04,04-05,05-06,06-07,07-08,08-09,09-10,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
Unidad de Programación,Tipo Oferta,Nº contrato,,,,,,,,,,,,,,,,,,,,,
ACSOC01,8,2357,-1.9,-1.9,-1.9,-1.9,-2.0,-2.1,-2.6,-3.5,-4.3,-4.5,...,-3.8,-3.7,-3.5,-3.3,-3.2,-3.1,-3.0,-2.7,-2.5,-2.3
ADSEC01,8,2354,-0.5,-0.4,-0.4,-0.3,-0.3,-0.4,-0.4,-0.5,-0.6,-0.6,...,-0.6,-0.6,-0.6,-0.6,-0.6,-0.7,-0.8,-0.7,-0.7,-0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WKWC01,8,2346,-1.7,-1.5,-1.3,-1.3,-1.3,-1.3,-1.4,-1.8,-2.1,-2.3,...,-2.1,-2.0,-2.0,-2.1,-2.3,-2.7,-2.8,-2.6,-2.4,-2.0
YELLC01,8,2001,-112.4,-109.4,-108.1,-108.4,-108.9,-116.5,-151.4,-187.8,-216.4,-218.5,...,-172.4,-173.4,-179.9,-186.9,-184.3,-181.2,-171.0,-157.0,-142.5,-133.3


## Data preprocessing

In [2]:
import numpy as np

df = df_base.melt(ignore_index=False).reset_index()
df.columns = ['unit', 'offer_type', 'contract', 'hour', 'value']

date = '20240221'
hour = df.hour.str.split('-', expand=True)[0]
df['datetime'] = pd.to_datetime(date + ' ' + hour)

df['agent'] = np.where(df.value > 0, 'Buyer', 'Seller')
df

,unit,offer_type,contract,hour,value,datetime,agent
0,ACSOC01,8,2357,00-01,-1.9,2024-02-21 00:00:00,Seller
1,ADSEC01,8,2354,00-01,-0.5,2024-02-21 00:00:00,Seller
...,...,...,...,...,...,...,...
12262,WKWC01,8,2346,23-24,-2.0,2024-02-21 23:00:00,Seller
12263,YELLC01,8,2001,23-24,-133.3,2024-02-21 23:00:00,Seller


## Exploratory data analysis

### Units involved in contracts

In [3]:
df_base = df.copy()

In [4]:
df_base.pivot_table(index='unit', columns='contract', values='value', aggfunc='sum').iloc[:20,:20].style.format(precision=0).highlight_null(props='color: transparent')

contract,1019,1513,1734,1771,1785,1793,1873,1944,1945,1948,1963,1991,1993,1994,1998,2001,2011,2012,2018,2019
unit,,,,,,,,,,,,,,,,,,,,
ACSOC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ADSEC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ADXC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ALBCC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4,6
ALSEC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ALUZC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ALZ1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,342,nan,nan,nan,nan,nan,nan
ALZ2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,342,nan,nan,nan,nan,nan,nan
APRIC01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### Unit specific involved in contracts

In [5]:
unit = "ENDE01"
contracts = df_base.query('unit == @unit').contract.unique()
contracts

array([1019, 1513, 1994, 2293, 2193, 2330])

In [6]:
mask = df_base.contract.isin(contracts)
df = df_base[mask]
df

,unit,offer_type,contract,hour,value,datetime,agent
12,ALZ1,1,1994,00-01,NaN,2024-02-21 00:00:00,Seller
16,ALZ2,1,1994,00-01,NaN,2024-02-21 00:00:00,Seller
...,...,...,...,...,...,...,...
12214,TERE,1,1513,23-24,NaN,2024-02-21 23:00:00,Seller
12241,VAN2,1,1513,23-24,NaN,2024-02-21 23:00:00,Seller


In [7]:
df.groupby(['unit', 'agent']).value.sum().unstack('agent').sort_values('Buyer', ascending=False).fillna(0).style.format(precision=0)

agent,Buyer,Seller
unit,,
ASC1,2985,0
ASC2,2526,0
VAN2,2256,0
EBRFEN,1644,0
SBEU,1230,0
ENDE01,589,-12182
ALZ2,342,0
ALZ1,342,0
EREGPE1,149,0


## References

https://www.energychisquared.com/post/tipos-de-bilaterales-en-el-sistema-el%C3%A9ctrico/